# This is a sample Jupyter Notebook

Below is an example of a code cell.

Put your cursor into the cell and press **Shift + Enter** to execute it and select the next one,
or click the **Run Cell** button.

Press **Double Shift** to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see
[PyCharm Jupyter help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).

For an overview of PyCharm, go to **Help → Learn IDE features**
or refer to the [documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).


In [1]:
import os
from glob import glob
from dotenv import load_dotenv

load_dotenv()

PDF_FOLDER = "data/pdfs"

pdf_files = glob(os.path.join(PDF_FOLDER, "*.pdf"))

for f in pdf_files:
    print(f)

data/pdfs\Guerra1.pdf
data/pdfs\Guerra2.pdf
data/pdfs\Guerra3.pdf


In [2]:
from langchain_community.document_loaders import PyPDFLoader

docs = []

for pdf_path in pdf_files:
    loader = PyPDFLoader(pdf_path)
    pdf_docs = loader.load()
    docs.extend(pdf_docs)

print(f"Total de páginas cargadas: {len(docs)}")

Total de páginas cargadas: 22


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=200,
    add_start_index=True
)

all_splits = text_splitter.split_documents(docs)

print(f"Chunks totales: {len(all_splits)}")

Chunks totales: 69


In [4]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="nomic-embed-text"
)

In [5]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="pdf_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db"
)

In [6]:
ids = vector_store.add_documents(all_splits)

print(f"Chunks indexados: {len(ids)}")

Chunks indexados: 69


In [7]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 5,
        "score_threshold": 0.2
    }
)


In [8]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3"
)

In [9]:
def ask_question(query):

    docs = retriever.invoke(query)

    context = "\n\n".join([doc.page_content for doc in docs])




    prompt = f"""
    Eres un asistente histórico experto en la Segunda Guerra Mundial.

    Responde a la pregunta del usuario utilizando SOLO la información proporcionada en el CONTEXTO.

    Si la respuesta NO aparece directamente en el contexto, pero puedes inferirla lógicamente de la información disponible, hazlo indicando: "Basado en los documentos, ..."

    Si la información no está en el contexto en absoluto, responde: "No encuentro información sobre esto en los documentos proporcionados."

    CONTEXTO:
    {context}

    PREGUNTA:
    {query}

    RESPUESTA:
    """


    response = llm.invoke(prompt)

    return response.content

In [10]:
ask_question("En que años se desarrolló la segunda guerra mundial?")


'La respuesta a tu pregunta está disponible directamente en el contexto.\n\nSegún el texto, la Segunda Guerra Mundial se desarrolló desde 1939 hasta 1945.'